<a href="https://colab.research.google.com/github/Keerthu8999/GoogleColab/blob/main/ProjectNull.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import spacy
from spacy.training import Example
from spacy.scorer import Scorer
from sklearn.metrics import classification_report
from CoNLL2Spacy import *
import random


In [3]:
def load_data():
    file = open("/content/Crimetest.txt", "r",encoding = "utf-8")
    valList = []
    for line in file:
        valList.append(line[:-1])
    test_data = conll2spacy(valList)  # Your testing data
    file = open("/content/Crimetrain.txt", "r",encoding = "utf-8")
    trainList = []
    for line in file:
        trainList.append(line[:-1])
    train_data = conll2spacy(trainList)
    return train_data, test_data


In [9]:
def train_spacy(train_data, test_data, iterations=20, dropout=0.5, model_name="modelTrained"):
    nlp = spacy.blank("en")  # Load English language model
    ner = nlp.add_pipe("ner", last=True)  # Add Named Entity Recognition pipeline

    # Add labels to the NER pipeline
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # Convert training data to spaCy Example objects
    train_examples = []
    for text, annotations in train_data:
        train_examples.append(Example.from_dict(nlp.make_doc(text), annotations))

    # Disable other pipelines during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            random.shuffle(train_examples)
            losses = {}
            for example in train_examples:
                nlp.update([example], drop=dropout, losses=losses)
            print("Iteration:", itn, "Losses:", losses)

    # Save the trained model
    nlp.to_disk(model_name)

In [5]:
def evaluate_spacy(model_name, test_data):
    nlp = spacy.load(model_name)  # Load trained model
    scorer = Scorer()
    true_labels = []
    pred_labels = []
    for text, annotations in test_data:
        doc = nlp(text)
        true_entities = [ent for _, ent in annotations.get("entities")]
        pred_entities = [ent.label_ for ent in doc.ents]
        true_labels.extend(true_entities)
        pred_labels.extend(pred_entities)
        example = Example.from_dict(doc, annotations)
        scorer.score([example])  # Update scorer with the current example
    print(classification_report(true_labels, pred_labels))

In [6]:
def main():
    train_data, test_data = load_data()
    train_spacy(train_data, test_data)
    evaluate_spacy("modelTrained", test_data)

In [ ]:
if __name__ == "__main__":
    main()

Iteration: 0 Losses: {'ner': 81852.24288907254}
Iteration: 1 Losses: {'ner': 64739.57049821249}
